# YELP - REVIEW

In [1]:
import pandas as pd
from pandas import json_normalize
import os
import re
import json 
# pip install nltk
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\ALEX\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [62]:
def cargar_json_en_dataframe(archivo_json, tamaño_lote=1000):
    json_reader = pd.read_json(archivo_json, lines=True, chunksize=tamaño_lote)
    dataframe_list = []

    for chunk in json_reader:
        dataframe_list.append(chunk)

    dataframe = pd.concat(dataframe_list, ignore_index=True)
    return dataframe

# Uso del método
archivo_json = 'Yelp/review.json'
tamaño_lote = 1000
dataframe = cargar_json_en_dataframe(archivo_json, tamaño_lote)

dataframe  

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3,0,0,0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11
1,BiTunyQ73aT9WBnpR9DZGw,OyoGAe7OKpv6SyGZT5g77Q,7ATYjTIgM3jUlt4UM3IypQ,5,1,0,1,I've taken a lot of spin classes over the year...,2012-01-03 15:28:18
2,saUsX_uimxRlCVr67Z4Jig,8g_iMtfSiwikVnbP2etR0A,YjUWPpI6HXG530lwP-fb2A,3,0,0,0,Family diner. Had the buffet. Eclectic assortm...,2014-02-05 20:30:30
3,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5,1,0,1,"Wow! Yummy, different, delicious. Our favo...",2015-01-04 00:01:03
4,Sx8TMOWLNuJBWer-0pcmoA,bcjbaE6dDog4jkNY91ncLQ,e4Vwtrqf-wpJfwesgvdgxQ,4,1,0,1,Cute interior and owner (?) gave us tour of up...,2017-01-14 20:54:15
...,...,...,...,...,...,...,...,...,...
6990275,H0RIamZu0B0Ei0P4aeh3sQ,qskILQ3k0I_qcCMI-k6_QQ,jals67o91gcrD4DC81Vk6w,5,1,2,1,Latest addition to services from ICCU is Apple...,2014-12-17 21:45:20
6990276,shTPgbgdwTHSuU67mGCmZQ,Zo0th2m8Ez4gLSbHftiQvg,2vLksaMmSEcGbjI5gywpZA,5,2,1,2,"This spot offers a great, affordable east week...",2021-03-31 16:55:10
6990277,YNfNhgZlaaCO5Q_YJR4rEw,mm6E4FbCMwJmb7kPDZ5v2Q,R1khUUxidqfaJmcpmGd4aw,4,1,0,0,This Home Depot won me over when I needed to g...,2019-12-30 03:56:30
6990278,i-I4ZOhoX70Nw5H0FwrQUA,YwAMC-jvZ1fvEUum6QkEkw,Rr9kKArrMhSLVE9a53q-aA,5,1,0,0,For when I'm feeling like ignoring my calorie-...,2022-01-19 18:59:27


## ETL review Yelp

In [63]:
dataframe['date'] = pd.to_datetime(dataframe['date'])

In [64]:
# Agrega una nueva columna 'Año' al DataFrame
dataframe['year'] = dataframe['date'].dt.year

In [65]:
df_review_filtro = dataframe[dataframe['year'] >= 2015]
print(df_review_filtro.shape)
df_review_filtro.head()

(5285402, 10)


,review_id,user_id,business_id,stars,useful,funny,cool,text,date,year
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3,0,0,0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11,2018
3,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5,1,0,1,"Wow! Yummy, different, delicious. Our favo...",2015-01-04 00:01:03,2015
4,Sx8TMOWLNuJBWer-0pcmoA,bcjbaE6dDog4jkNY91ncLQ,e4Vwtrqf-wpJfwesgvdgxQ,4,1,0,1,Cute interior and owner (?) gave us tour of up...,2017-01-14 20:54:15,2017
5,JrIxlS1TzJ-iCu79ul40cQ,eUta8W_HdHMXPzLBBZhL1A,04UD14gamNjLY0IDYVhHJg,1,1,2,1,I am a long term frequent customer of this est...,2015-09-23 23:10:31,2015
6,6AxgBCNX_PNTOxmbRSwcKQ,r3zeYsv1XFBRA4dJpL78cw,gmjsEdUsKpj9Xxu6pdjH0g,5,0,2,0,Loved this tour! I grabbed a groupon and the p...,2015-01-03 23:21:18,2015


In [66]:
df1 = df_review_filtro.copy()

In [1]:
import pandas as pd

In [2]:
df_bus = pd.read_csv("IDs/ids_yelp_business.csv")
df_bus.shape

(253, 3)

In [4]:
# Suponiendo que 'gmap_id' es el nombre de la columna
lista_gmap_id = df_bus['business_id'].tolist()

# Imprimir la lista de gmap_id
print(lista_gmap_id)
len(lista_gmap_id)

['VZzivJEbmHItOxNXJB3SpA', 'vuL3QCjFZ7lU5LMdUAmZ9Q', 'gi90NMmRhzl8vzM83UNOVA', 'xxxL7I3hGCEymNlYLXUyBA', 'wXBAAafoiZ4Nty2ajuBtfw', 'CmDTNEVReN6CgE6ctlZhag', 'yDriUh65Wpx1HnTmXln9nw', 'KE_oM4y1GMI3YMOnnqHqRQ', '31s1x27DnN2V-ptUWEdfTQ', '1QtF1bFgzG3Jo0-L4kv0Cg', 'HZtkTPfSshfRjpQR7x1Raw', 'zJdQwRbI7p0bH_60sfBSYA', '4KPEgM1E7WgQQqd0IAIopg', 'qpIpYSzySrqNmPWEPNplug', 'BQdg19sukvw90QJ4tbY-BA', 'DfkO7MHuy2wxgeUZ2-o3BA', 'qBvsNlxtqy8hPxx1C7HuVA', 'cdPjDZubPvok0VHv654Tvw', 'XF86J5bJKkmueBL-PmPbcg', '6Kc2FhLppRhWBXDKxjxj2A', 'vLdELbSYtwKVpkl4ownLEA', 'yheGRZWHC-RJoKlqepj0hw', 'NFdlTTZrmXejfKyz7v5T3Q', 'swKXtO22QjVIZTBU8UP7Pg', 'M2S5ELqHjz1B1booBTN1dQ', '67QczBNcTAYq8alxqwiZ5g', 'd-hMYlJ-hMAHSgd3KvYcWg', 'EwblShAzTJ4Op6aJZYYuVA', 'wQnVwt98pXX0n0Szro_XGg', '7ALZQc35zGG0a1SwSnExJg', 'AnS7cUS7RzXnEBx6c7y5JA', '_AvFiC-whJC9g9ndd3XGtA', 'Rx6eQL9PdxJK8wgCbKDu-A', 'wgDanVXxbMuP3djeFCk4PQ', 'q2rNovrR1ygAr5TLHSX70g', 'GwUfmvUOAWmByr0rRn2aTA', 'VqXABZYXe54vr5kpsKbFsg', 'ybgLYF_SpBTmlWqa3S3lcw', 'BQVgKbITz9

253

In [68]:
# Filtramos reviews de solo los locales que estudiamos
df1 = df1[df1['business_id'].isin(df_bus['business_id'])]
df1 # 1832

,review_id,user_id,business_id,stars,useful,funny,cool,text,date,year
9865,4wNMdKSqUUzfoXv6-Q1DWQ,WisRT3bV_ed2BuabSPKyjw,EwblShAzTJ4Op6aJZYYuVA,1,0,0,0,Terrible service. Went at 1pm on a Saturday an...,2017-05-13 19:06:28,2017
9880,TcxmUZDm1NbA94__c8V-8Q,o0wXgcns8Ch7h9rm_rkDtA,cdPjDZubPvok0VHv654Tvw,1,0,0,0,Abysmal customer service. Came in on Wednesday...,2016-11-09 23:24:14,2016
10188,MdsSdcnMLU89IEkHxkNofg,Ug6Djz3endu6XHg7aSh9yw,BQdg19sukvw90QJ4tbY-BA,4,0,0,0,"Standard Subway, nothing special. Fast, friend...",2015-01-19 01:54:27,2015
11271,1gpg75fdwRnFIfRYvCNHUg,EWmxCRMYEO3vaHXhJ35smg,XF86J5bJKkmueBL-PmPbcg,5,0,0,0,I have been here a bunch of times.. It's alway...,2016-04-13 11:41:49,2016
12556,CEHaucuAzqSzOdMB3mPNTA,jsej79maZJg_X619aw8dig,1QtF1bFgzG3Jo0-L4kv0Cg,2,1,0,0,This place was slow and didn't have any deals ...,2017-10-24 21:40:04,2017
...,...,...,...,...,...,...,...,...,...,...
6973271,Eh7V3JK67khKMcuB7SjzbQ,sTe-Vnzl9Of4NyMNW5UfMw,k0PzT0_WiOZXndrsz8fIxw,1,2,0,0,Service is pretty bad. Meatball sub was the sl...,2018-10-24 19:10:21,2018
6975150,AYxvtRaAtqoOWV2NkTbxkw,6sW_A0JeTbOUsRRnR6D9vw,-x-niT6JolhXi6VuKclgxw,1,0,0,0,"Terrible subway. Awful counter staff, they sho...",2017-02-11 14:29:12,2017
6975987,4oS2vHQm572DY_S1L6vUcQ,uzZki4e4aNzYXCU7B__uFQ,hwRGsA-nW5Hi9LTApwA5RA,5,0,0,0,"Tonight I went into this subway at 8:25, right...",2022-01-13 03:49:40,2022
6977759,rm5wF9XeIbL8JMYDM-yYxw,skdRkQKarDESZ5IoWjWQvw,L617WGpANJZW8orvCdOO1w,1,0,0,0,I used to work at this location a few weeks ag...,2020-01-30 20:31:53,2020


In [69]:
# Convierte la columna 'Date' a tipo datetime y Extrae solo la parte de la fecha
df1.loc[:, 'date'] = pd.to_datetime(df1['date']).dt.date
print(df1.shape)
df1.head(2)

(1832, 10)


,review_id,user_id,business_id,stars,useful,funny,cool,text,date,year
9865,4wNMdKSqUUzfoXv6-Q1DWQ,WisRT3bV_ed2BuabSPKyjw,EwblShAzTJ4Op6aJZYYuVA,1,0,0,0,Terrible service. Went at 1pm on a Saturday an...,2017-05-13,2017
9880,TcxmUZDm1NbA94__c8V-8Q,o0wXgcns8Ch7h9rm_rkDtA,cdPjDZubPvok0VHv654Tvw,1,0,0,0,Abysmal customer service. Came in on Wednesday...,2016-11-09,2016


Análisis de sentimiento

In [70]:
# Convierte a minuscula y remueve caracteres especiales

def limpiar_texto(texto):
    if isinstance(texto, str):
        texto = texto.lower()
        texto = re.sub(r'[^a-z0-9\s]', '', texto)
    return texto

df1['text'] = df1['text'].apply(limpiar_texto)

In [71]:
# Inicializar SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()

# Función para realizar el análisis de sentimientos y asignar categorías
def categorize_sentiment(review):
    # Verifica si la reseña está ausente
    if pd.isnull(review) or not isinstance(review, str):
        return 1  # Valor por defecto si la reseña está ausente
    else:
        # Realiza análisis de sentimientos con SentimentIntensityAnalyzer
        sentiment_score = sia.polarity_scores(review)['compound']
        
        # Asigna categoría según la escala proporcionada
        if sentiment_score <= 0:
            return 0  # Negativo
        else:
            return 1  # Positivo

In [72]:
# Aplicar la función a la columna 'review' y crear una nueva columna 'sentiment_category'
df1['sentiment_analysis'] = df1['text'].apply(categorize_sentiment)

In [73]:
# Convierte la columna 'sentiment_analysis' a tipo de datos object
df1['sentiment_analysis'] = df1['sentiment_analysis'].astype('object')

# Reemplaza 'SD' en la columna 'sentiment_analysis' donde 'text' sea None
df1.loc[df1['text'].isnull(), 'sentiment_analysis'] = 'SD'

In [82]:
df1['sentiment_analysis'].value_counts()

sentiment_analysis
1    981
0    851
Name: count, dtype: int64

In [81]:
df1['sentiment_analysis'] = df1['sentiment_analysis'].astype(str)

In [83]:
print(df1.shape)
df1.head(2)

(1832, 11)


,review_id,user_id,business_id,stars,useful,funny,cool,text,date,year,sentiment_analysis
9865,4wNMdKSqUUzfoXv6-Q1DWQ,WisRT3bV_ed2BuabSPKyjw,EwblShAzTJ4Op6aJZYYuVA,1,0,0,0,terrible service went at 1pm on a saturday and...,2017-05-13,2017,0
9880,TcxmUZDm1NbA94__c8V-8Q,o0wXgcns8Ch7h9rm_rkDtA,cdPjDZubPvok0VHv654Tvw,1,0,0,0,abysmal customer service came in on wednesday ...,2016-11-09,2016,0


Guardar archivo

In [79]:
df1.to_csv("yelp_review.csv", index = False)

In [46]:
df_usuarios_que_comentan = df1[['user_id']].copy()

In [47]:
df_usuarios_que_comentan.to_csv("yelp_review_usuariosquecomentan.csv", index = False)

In [2]:
df_usuarios_que_comentan = pd.read_csv('yelp_review_usuariosquecomentan.csv')

# YELP - USER

In [30]:
#pip install --upgrade pandas pyarrow
import pandas as pd

In [31]:
dff = pd.read_parquet("Yelp/user.parquet") 
print(dff.shape)
dff

(2105597, 22)


,user_id,name,review_count,yelping_since,useful,funny,cool,elite,friends,fans,...,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
0,qVc8ODYU5SZjKXVBgXdI7w,Walker,585,2007-01-25 16:47:26,7217,1259,5994,2007,"NSCy54eWehBJyZdG2iE84w, pe42u7DcCH2QmI81NX-8qA...",267,...,65,55,56,18,232,844,467,467,239,180
1,j14WgRoU_-2ZE1aw1dXrJg,Daniel,4333,2009-01-25 04:35:42,43091,13066,27281,"2009,2010,2011,2012,2013,2014,2015,2016,2017,2...","ueRPE0CX75ePGMqOFVj6IQ, 52oH4DrRvzzl8wh5UXyU0A...",3138,...,264,184,157,251,1847,7054,3131,3131,1521,1946
2,2WnXYQFK0hXEoTxPtV2zvg,Steph,665,2008-07-25 10:41:00,2086,1010,1003,"2009,2010,2011,2012,2013","LuO3Bn4f3rlhyHIaNfTlnA, j9B4XdHUhDfTKVecyWQgyA...",52,...,13,10,17,3,66,96,119,119,35,18
3,SZDeASXq7o05mMNLshsdIA,Gwen,224,2005-11-29 04:38:33,512,330,299,"2009,2010,2011","enx1vVPnfdNUdPho6PH_wg, 4wOcvMLtU6a9Lslggq74Vg...",28,...,4,1,6,2,12,16,26,26,10,9
4,hA5lMy-EnncsH4JoR-hFGQ,Karen,79,2007-01-05 19:40:59,29,15,7,,"PBK4q9KEEBHhFvSXCUirIw, 3FWPpM7KU1gXeOM_ZbYMbA...",1,...,1,0,0,0,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2105592,4QGxxakRZeOlg_qDuxmTeQ,Jennilee,38,2012-01-19 23:33:02,74,9,6,,kmwNG5LZSHFmveg6wYYdrw,0,...,1,0,0,0,1,4,0,0,1,0
2105593,tmelBbVBGAzXBVfH2u_R6g,Gerry,19,2009-06-09 16:34:54,14,5,2,,"BFYdCAMFyjYHDwesndEXEg, _9fTIqfSJc7g3V_o76XRVg...",1,...,1,0,0,0,0,1,0,0,0,0
2105594,tpBznnD6uJN3m_pJubj09w,Emily,26,2013-08-13 23:18:11,4,1,2,,"bKV3ly2MuK-K1cptMrFknQ, liel18zRoSB4tEkUP7i6Cg...",0,...,0,0,0,0,1,0,0,0,0,0
2105595,Kst_srPw7GdYydMFYdCtzw,Heatheranne,25,2015-01-10 00:06:25,21,2,5,,"dzHTk52vbGtbktRm_B-wEg, fOfFLV7IbBDN6lzARaLqdg...",0,...,0,0,0,0,0,1,0,0,0,0


In [33]:
dff.columns

Index(['user_id', 'name', 'review_count', 'yelping_since', 'useful', 'funny',
       'cool', 'elite', 'friends', 'fans', 'average_stars', 'compliment_hot',
       'compliment_more', 'compliment_profile', 'compliment_cute',
       'compliment_list', 'compliment_note', 'compliment_plain',
       'compliment_cool', 'compliment_funny', 'compliment_writer',
       'compliment_photos'],
      dtype='object')

In [16]:
dff.to_csv("yelp_user_i.csv", index = False)

In [34]:
df_user_1 = dff.iloc[:, :11]
df_user_1

,user_id,name,review_count,yelping_since,useful,funny,cool,elite,friends,fans,average_stars
0,qVc8ODYU5SZjKXVBgXdI7w,Walker,585,2007-01-25 16:47:26,7217,1259,5994,2007,"NSCy54eWehBJyZdG2iE84w, pe42u7DcCH2QmI81NX-8qA...",267,3.91
1,j14WgRoU_-2ZE1aw1dXrJg,Daniel,4333,2009-01-25 04:35:42,43091,13066,27281,"2009,2010,2011,2012,2013,2014,2015,2016,2017,2...","ueRPE0CX75ePGMqOFVj6IQ, 52oH4DrRvzzl8wh5UXyU0A...",3138,3.74
2,2WnXYQFK0hXEoTxPtV2zvg,Steph,665,2008-07-25 10:41:00,2086,1010,1003,"2009,2010,2011,2012,2013","LuO3Bn4f3rlhyHIaNfTlnA, j9B4XdHUhDfTKVecyWQgyA...",52,3.32
3,SZDeASXq7o05mMNLshsdIA,Gwen,224,2005-11-29 04:38:33,512,330,299,"2009,2010,2011","enx1vVPnfdNUdPho6PH_wg, 4wOcvMLtU6a9Lslggq74Vg...",28,4.27
4,hA5lMy-EnncsH4JoR-hFGQ,Karen,79,2007-01-05 19:40:59,29,15,7,,"PBK4q9KEEBHhFvSXCUirIw, 3FWPpM7KU1gXeOM_ZbYMbA...",1,3.54
...,...,...,...,...,...,...,...,...,...,...,...
2105592,4QGxxakRZeOlg_qDuxmTeQ,Jennilee,38,2012-01-19 23:33:02,74,9,6,,kmwNG5LZSHFmveg6wYYdrw,0,2.98
2105593,tmelBbVBGAzXBVfH2u_R6g,Gerry,19,2009-06-09 16:34:54,14,5,2,,"BFYdCAMFyjYHDwesndEXEg, _9fTIqfSJc7g3V_o76XRVg...",1,3.68
2105594,tpBznnD6uJN3m_pJubj09w,Emily,26,2013-08-13 23:18:11,4,1,2,,"bKV3ly2MuK-K1cptMrFknQ, liel18zRoSB4tEkUP7i6Cg...",0,3.85
2105595,Kst_srPw7GdYydMFYdCtzw,Heatheranne,25,2015-01-10 00:06:25,21,2,5,,"dzHTk52vbGtbktRm_B-wEg, fOfFLV7IbBDN6lzARaLqdg...",0,2.85


In [35]:
df_user_1 = df_user_1.drop(columns=['fans','elite','useful','funny','cool'])

In [36]:
df_user_1

,user_id,name,review_count,yelping_since,friends,average_stars
0,qVc8ODYU5SZjKXVBgXdI7w,Walker,585,2007-01-25 16:47:26,"NSCy54eWehBJyZdG2iE84w, pe42u7DcCH2QmI81NX-8qA...",3.91
1,j14WgRoU_-2ZE1aw1dXrJg,Daniel,4333,2009-01-25 04:35:42,"ueRPE0CX75ePGMqOFVj6IQ, 52oH4DrRvzzl8wh5UXyU0A...",3.74
2,2WnXYQFK0hXEoTxPtV2zvg,Steph,665,2008-07-25 10:41:00,"LuO3Bn4f3rlhyHIaNfTlnA, j9B4XdHUhDfTKVecyWQgyA...",3.32
3,SZDeASXq7o05mMNLshsdIA,Gwen,224,2005-11-29 04:38:33,"enx1vVPnfdNUdPho6PH_wg, 4wOcvMLtU6a9Lslggq74Vg...",4.27
4,hA5lMy-EnncsH4JoR-hFGQ,Karen,79,2007-01-05 19:40:59,"PBK4q9KEEBHhFvSXCUirIw, 3FWPpM7KU1gXeOM_ZbYMbA...",3.54
...,...,...,...,...,...,...
2105592,4QGxxakRZeOlg_qDuxmTeQ,Jennilee,38,2012-01-19 23:33:02,kmwNG5LZSHFmveg6wYYdrw,2.98
2105593,tmelBbVBGAzXBVfH2u_R6g,Gerry,19,2009-06-09 16:34:54,"BFYdCAMFyjYHDwesndEXEg, _9fTIqfSJc7g3V_o76XRVg...",3.68
2105594,tpBznnD6uJN3m_pJubj09w,Emily,26,2013-08-13 23:18:11,"bKV3ly2MuK-K1cptMrFknQ, liel18zRoSB4tEkUP7i6Cg...",3.85
2105595,Kst_srPw7GdYydMFYdCtzw,Heatheranne,25,2015-01-10 00:06:25,"dzHTk52vbGtbktRm_B-wEg, fOfFLV7IbBDN6lzARaLqdg...",2.85


In [37]:
df_user_1['yelping_since'] = pd.to_datetime(df_user_1['yelping_since'])
# Convierte la columna 'Date' a tipo datetime y Extrae solo la parte de la fecha
df_user_1.loc[:, 'yelping_since'] = pd.to_datetime(df_user_1['yelping_since']).dt.date

In [38]:
df_user_1.head(2)

,user_id,name,review_count,yelping_since,friends,average_stars
0,qVc8ODYU5SZjKXVBgXdI7w,Walker,585,2007-01-25,"NSCy54eWehBJyZdG2iE84w, pe42u7DcCH2QmI81NX-8qA...",3.91
1,j14WgRoU_-2ZE1aw1dXrJg,Daniel,4333,2009-01-25,"ueRPE0CX75ePGMqOFVj6IQ, 52oH4DrRvzzl8wh5UXyU0A...",3.74


In [39]:
df_user_1['user_id'].duplicated().sum()

117700

In [40]:
# Visualizar filas duplicadas
filas_duplicadas_completas = df_user_1[df_user_1.duplicated(keep=False)]
filas_duplicadas_completas = filas_duplicadas_completas.sort_values(by=list(df_user_1.columns))
filas_duplicadas_completas

,user_id,name,review_count,yelping_since,friends,average_stars
27105,---2PmXbF47D870stH1jqA,Susan,36,2012-10-24,"o9QkuHIyxmqbORF7PeAbuw, Q70zvjAa9aawgW7KsNVGDA...",4.98
2017202,---2PmXbF47D870stH1jqA,Susan,36,2012-10-24,"o9QkuHIyxmqbORF7PeAbuw, Q70zvjAa9aawgW7KsNVGDA...",4.98
2038302,---2PmXbF47D870stH1jqA,Susan,36,2012-10-24,"o9QkuHIyxmqbORF7PeAbuw, Q70zvjAa9aawgW7KsNVGDA...",4.98
2059402,---2PmXbF47D870stH1jqA,Susan,36,2012-10-24,"o9QkuHIyxmqbORF7PeAbuw, Q70zvjAa9aawgW7KsNVGDA...",4.98
2080502,---2PmXbF47D870stH1jqA,Susan,36,2012-10-24,"o9QkuHIyxmqbORF7PeAbuw, Q70zvjAa9aawgW7KsNVGDA...",4.98
...,...,...,...,...,...,...
2048241,zziJLt25YU6dp01sewR-IQ,Robin,222,2010-05-31,"LFBGuGPkVFkOyBPL020yxQ, UDyU9Va2MfulJXMaheiKoQ...",3.51
2069341,zziJLt25YU6dp01sewR-IQ,Robin,222,2010-05-31,"LFBGuGPkVFkOyBPL020yxQ, UDyU9Va2MfulJXMaheiKoQ...",3.51
2090441,zziJLt25YU6dp01sewR-IQ,Robin,222,2010-05-31,"LFBGuGPkVFkOyBPL020yxQ, UDyU9Va2MfulJXMaheiKoQ...",3.51
85855,zzljYL9c22HWRO3VvJ1_Pg,Barry,1,2015-07-06,"ak5D9IvG7Dgi8ZekffI4HQ, dBTHZ-SU55clkbmZ_A6xCg...",1.00


In [53]:
# Eliminar filas duplicadas completas y conserva la primera aparición
df_user_2 = df_user_1.drop_duplicates()
print(df_user_2.shape)

(1987897, 6)


In [54]:
df_user_2['user_id'].duplicated().sum()

0

In [55]:
print(df_user_2.shape)
df_user_2.head(2)

(1987897, 6)


,user_id,name,review_count,yelping_since,friends,average_stars
0,qVc8ODYU5SZjKXVBgXdI7w,Walker,585,2007-01-25,"NSCy54eWehBJyZdG2iE84w, pe42u7DcCH2QmI81NX-8qA...",3.91
1,j14WgRoU_-2ZE1aw1dXrJg,Daniel,4333,2009-01-25,"ueRPE0CX75ePGMqOFVj6IQ, 52oH4DrRvzzl8wh5UXyU0A...",3.74


In [56]:
df_user_2['review_count'].value_counts()

review_count
1       345437
2       243744
3       179060
4       132508
5       102241
         ...  
1694         1
2481         1
1784         1
1451         1
1197         1
Name: count, Length: 1973, dtype: int64

In [7]:
df_usuarios_que_comentan.head()

,user_id
0,WisRT3bV_ed2BuabSPKyjw
1,o0wXgcns8Ch7h9rm_rkDtA
2,Ug6Djz3endu6XHg7aSh9yw
3,EWmxCRMYEO3vaHXhJ35smg
4,jsej79maZJg_X619aw8dig


In [6]:
df_usuarios_que_comentan.shape
# Suponiendo que 'gmap_id' es el nombre de la columna
lista_user_id = df_usuarios_que_comentan['user_id'].tolist()

# Imprimir la lista de gmap_id
print(lista_user_id)
len(lista_user_id)

['WisRT3bV_ed2BuabSPKyjw', 'o0wXgcns8Ch7h9rm_rkDtA', 'Ug6Djz3endu6XHg7aSh9yw', 'EWmxCRMYEO3vaHXhJ35smg', 'jsej79maZJg_X619aw8dig', 'j6uHqtee4d8oonJTU4c1eg', 'Cm6c4A1Pz_Vwlkr09frWGw', 'fVup-p6mgIMuWzNIj-CS1Q', 'rBeKFxV0PJ-wXMjaEa8G5A', '9ineN7FeC3hfTByKbLy4VQ', 'LaNkSxqD1jbuTFKF_dBqOg', 'YjS4Ab5aGnGZ8xhuAJi37w', 'Lw6rx2yyzUbnFMUiD-Mk3Q', '98Mthgp9sLgynCB5d2Ngpg', 'LVx0Iyn_MfO9eFhl-uppNA', 'KXDiP5LvmOIAH2M7rQyquw', 'UgXTZwjh8AUali74U0ubMw', 'FJ7gVT8uFuOBYA9VquL14Q', 'tTWBzI0x_Mm8RsJgWb8OAg', 'fVup-p6mgIMuWzNIj-CS1Q', 'xYEdpNyQ8lBuQbZmAPJ_nA', 'I6chm-B8jTr48Ig5EuSynA', 'uZ_F1KUH5V04eMBYEXr6Xw', 'YjS4Ab5aGnGZ8xhuAJi37w', '94hrjl5NGeLbBCr08BHNBQ', 'dWTrJdEPoZB8LkOPv_jxsA', 'YazB4ROLMQ6-jvDfumvijA', 'Xb_fWQm_B4w0W4UW2qST2Q', 'PA7nyFeTM4xHs43Sb_Z41A', 'aRF-8E_9t0zpvryDPLMLBQ', 'moCR7JcRYbJvosBQjpgb7Q', '17dB77sQqIjRi5jhgsAicQ', 'UKDj7CGKHSp4svIRrFsmiA', 'qEwL-Duqr8Lwj5Mqa9_Y7A', 'eJfMuQ3kL5hNe6pLD0AHFw', 'KG8SNh9SITNPrGa6IdQcsg', 'g5cwn9zljtr7FyUy4k-9Bg', 'Tuatg01hL_1iB_JwKypZWw', 'DVH6DSdLgN

1832

In [57]:
# Filtramos reviews
df_user_2 = df_user_2[df_user_2['user_id'].isin(df_usuarios_que_comentan['user_id'])]
df_user_2 # 1716

,user_id,name,review_count,yelping_since,friends,average_stars
1614,mKBl4fAqTfNts7B78aOPVg,Dave,752,2009-10-11,"TrL_9PA5LSDXgCYU8I7NCg, vnl8aTXiLC08mVtG5Tu_xw...",3.68
2427,z33x0fHD8gutPy5Gv6Hwpw,Angela,484,2007-03-01,"SSX5Xc0KhobY7yL7G8SLtA, NXUdf0i5qOBo45JgXPMBTQ...",3.97
2608,1BjVegpTTmGgRm4x0l_hEw,Erich,39,2011-02-07,"IH-poLs_wMjX2wOZL9Kh8Q, Bn0558-nCOcLf-vlAdJUJQ...",3.49
3756,_ntA5HPDzqBgwg7S3s7w7Q,Sabrina,848,2008-03-15,"DIciwuIFTUChQ0ZbwVeCHQ, B-s-8VUnuBjGTP3d01jsyw...",3.93
4746,EvJ1yXjsBqK_NH493WdA6A,Kate,171,2010-02-14,"7K4SzSIG9zUzBLCh0gHNjA, s5txUZuoj4gNHQ3UY9Dnrg...",4.19
...,...,...,...,...,...,...
1966183,_sUfx_eW4ClwQt8EW8L1rg,Andrew,5,2014-03-05,None,2.40
1970924,St-sJpuqvGCDzcf_EuB_vw,Becca,1,2016-04-08,None,1.00
1974833,onFGgE5lHik_soHcctW9vQ,Gary,1,2021-12-17,None,1.00
1981574,2xFiZo_i4SQi_4Iflc81ww,J,1,2016-03-13,None,1.00


In [59]:
df_user_2.to_parquet("yelp_user_o.parquet", index = False)

In [60]:
df_user_2.to_csv("yelp_user_o.csv", index = False)

In [103]:
#df_user_3.to_json("yelp_user.json", orient='records', lines=True)